<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод-по-разделу" data-toc-modified-id="Вывод-по-разделу-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод по разделу</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span><ul class="toc-item"><li><span><a href="#Вывод-по-разделу:" data-toc-modified-id="Вывод-по-разделу:-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод по разделу:</a></span></li></ul></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Вывод-по-разделу" data-toc-modified-id="Вывод-по-разделу-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод по разделу</a></span></li></ul></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод:</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

<font size = 3><b>Описание данных</b></font>
- <b>Признаки</b>: пол, возраст и зарплата застрахованного, количество членов его семьи.
- <b>Целевой признак</b>: количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

In [1]:
#импортируем необходимые библиотеки
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
#загрузим наш датасет
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
#посмотрим на данные
df.sample(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
2698,1,20.0,43300.0,0,0
4174,1,38.0,40700.0,1,0
939,1,35.0,36100.0,2,0
555,0,20.0,40300.0,1,0
3078,0,27.0,52500.0,2,0


In [4]:
#и посмотрим общую информацию о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


У нас 3 столбца формата `int64` и 2 столбца формата `float64`. Как видно, в столбцах `Возраст` и `Зарплата` после запятой идут нули, поэтому можем преобразовать эти столбцы также в целочисленный формат.

In [5]:
df = df.astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                5000 non-null   int64
 1   Возраст            5000 non-null   int64
 2   Зарплата           5000 non-null   int64
 3   Члены семьи        5000 non-null   int64
 4   Страховые выплаты  5000 non-null   int64
dtypes: int64(5)
memory usage: 195.4 KB


In [6]:
#посмотрим на количество дубликатов
print('Количество дубликатов:', df.duplicated().sum(),
      'что составляет', (df.duplicated().sum()/len(df))*100, 'процентов от датасета.')

Количество дубликатов: 153 что составляет 3.06 процентов от датасета.


Хотя и не совсем понятно, дубликаты это или совпадения, т.к. данные обезличены, но лучше эти дубликаты удалить, потому что их количество невелико и на результатах исследования это удаление никак не скажется.

In [7]:
df.drop_duplicates(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                4847 non-null   int64
 1   Возраст            4847 non-null   int64
 2   Зарплата           4847 non-null   int64
 3   Члены семьи        4847 non-null   int64
 4   Страховые выплаты  4847 non-null   int64
dtypes: int64(5)
memory usage: 227.2 KB


In [8]:
#теперь проверим датасет на пропуски
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Посмотрим на таблицу с описательной статистикой и есть ли какие-то аномалии в данных.

In [9]:
display(df.describe())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,4847.000000,4847.000000,4847.000000,4847.000000,4847.000000
mean,0.498453,31.023932,39895.811223,1.203425,0.152259
std,0.500049,8.487995,9972.952441,1.098664,0.468934
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33200.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Исходя из таблицы, в данных нет аномалий.
- В столбце "Пол" всего два значения, 0 и 1, что говорит о мужском и женском поле; 
- В столбце "Возраст" указан возраст от 18 до 65 лет, как раз минимальный и максимальный возраст возможности оформления большинства страховых продуктов.
- В столбце "Зарплата" значения от 5300 рублей до 79000 рублей, что говорит о том, что данные были выгружены либо много лет назад, либо здесь содержится информация в основном о небольших регионах с невысоким уровнем дохода.
- В столбце "Члены семьи" значения от 0 до 6, что тоже является абсолютно нормальными значениями.
- И в столбце "Страховые выплаты" указано количество обращений за выплатами за последние 5 лет и как мы видим, даже в третьем квантиле указано значение 0, среднее значение составляет всего 0,148, а максимальное значение составляет 5 обращений, что говорит о том, что большинство людей не обращается за страховыми выплатами.

In [10]:
#посмотрим на количество страховых выплат подробнее
df['Страховые выплаты'].value_counts()

0    4284
1     423
2     114
3      18
4       7
5       1
Name: Страховые выплаты, dtype: int64

### Вывод по разделу
1. В датасете 5000 строк и 5 столбцов.
2. Весь датасет был переведен в формат `int32`.
3. В датасете были обнаружены и впоследствии удалены 153 дубликата.
4. Пропусков в датасете нет.

<b>Наш датасет готов исследованию.</b>

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:**
признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:**
качество линейной регрессии не изменится.

**Обоснование:**

Создадим новую матрицу признаков из нашей старой матрицы $X$, умноженной на обратимую матрицу $P$:

$$
X_p = X * P
$$

Теперь создадим новый вектор весов, используя свойство транспонированных и обратимых матриц, а также единичную матрицу $E$:

$$
w_p = ((XP)^T XP)^{-1} (XP)^T y = (P^T X^T XP)^{-1} P^T X^T y = (P^T(X^T X)P)^{-1} P^T X^T y = P^{-1}(X^T X)^{-1}(P^T)^{-1}  P^T X^T y = = P^{-1}(X^T X)^{-1}EX^T y = P^{-1}(X^T X)^{-1}X^T y
$$

Итоговая формула получилась: $w_p = P^{-1}(X^T X)^{-1}X^T y$, в которой $(X^T X)^{-1}X^T y$ равно формуле обучения $w$.

Получилось, что $w_p = P^{-1}w$

Теперь построим предсказание по выведенной формуле: $a_p = X_p w_p = X P P^{-1} w$.

$P P^{-1}$ равно единичной матрице $E$ и получается, что $a_p = XEw = Xw$

И, следовательно, $a = a_p$

### Вывод по разделу:
При умножении признаков линейной регрессии на обратимую матрицу качество модели не изменится.

## Алгоритм преобразования

**Алгоритм**

Для защиты информации умножим матрицу признаков на обратимую матрицу $P$, которая генерируется случайным образом.

Этапы алгоритма:
1. Создание случайной квадратной обратимой матрицы с количеством столбцов, которое соответствует количеству признаков.
2. Проверка матрицы на обратимость.
3. Умножение матрицы признаков на случайную обратимую матрицу.
4. Применение модели на преобразованных признаках.
5. Сравнить метрики R2 на исходных признаках и на признаках, умноженных на обратимую матрицу.

**Обоснование**

Как мы выяснили выше, обратимая матрица не должна ухудшить качество модели.

## Проверка алгоритма

In [11]:
#выделим из датасета признаки и целевой признак
features= df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

In [12]:
#разделим на обучающую и тестовую выборки
(features_train, features_test, target_train, target_test) =\
train_test_split(features, target, test_size = 0.25, random_state=12345)
print(features_train.shape)
print(features_test.shape)

(3635, 4)
(1212, 4)


In [13]:
def score(features, target):
    model = LinearRegression()
    model.fit(features, target)
    predictions = model.predict(features)
    r2 = r2_score(target, predictions)
    return r2

In [14]:
print('R2_score стандартной модели:', score(features_train, target_train))

R2_score стандартной модели: 0.43215820265809746


Создадим случайную матрицу размера 4х4, потому что у нас 4 признака и обратимая матрица должна быть квадратной.

In [17]:
def rnd_matrix(n):
    random_matrix = np.random.normal(size=(4, 4))
    try:
        np.linalg.inv(random_matrix) # выполняется операция создания обратной матрицы
    except LinAlgError: # если возникает ошибка, т.е. обратной матрицы не существует, выполняется еще одна попытка
        rnd_matrix(n)
    return random_matrix

In [18]:
invert_matrix = rnd_matrix(10)

Матрица является обратимой. Теперь умножим матрицу с признаками на нашу случайную матрицу $P$.

In [19]:
features_train_matrix = features_train @ invert_matrix
features_test_matrix = features_test @ invert_matrix
print(features_train_matrix.shape)
print(features_test_matrix.shape)

(3635, 4)
(1212, 4)


In [20]:
#посмотрим, как выглядят преобразовынные матрицы
display(features_train_matrix.sample(5))
display(features_test_matrix.sample(5))

,0,1,2,3
687,-11571.768578,-65181.931521,47333.739449,-7784.717002
1372,-12329.572599,-69445.057848,50432.238462,-8295.994773
3156,-6604.854183,-37155.375270,26990.526536,-4446.209742
3921,-7282.698435,-40960.294214,29759.428680,-4905.313108
1355,-8626.927174,-48580.638826,35280.742207,-5804.425927


,0,1,2,3
1931,-8981.327088,-50556.926765,36723.187985,-6045.387801
540,-10515.452896,-59241.615355,43021.773785,-7074.871630
1502,-12729.600220,-71731.356579,52088.242012,-8563.342167
731,-9617.341032,-54218.539115,39369.130630,-6469.697980
2134,-8147.157972,-45835.470381,33295.217613,-5484.213545


Размеры выборок совпадают с изначальными. Проверим качество работы модели на новых признаках.

In [21]:
#проверим r2 на преобразованной обучающей выборке
print('R2_score на преобразованной обучающей выборке:', score(features_train_matrix, target_train))

R2_score на преобразованной обучающей выборке: 0.43215820265810256


In [22]:
#и проверим совпадает ли r2 на тестовых выборках
print('R2_score на тестовой выборке:', score(features_test, target_test))

R2_score на тестовой выборке: 0.425552801076332


In [23]:
print('R2_score на преобразованной тестовой выборке:', score(features_test_matrix, target_test))

R2_score на преобразованной тестовой выборке: 0.42555280107633287


Как видим, метрика R2 не изменилась и совпадает на обычной и преобразованной выборках.

### Вывод по разделу
Качество линейной регрессии не изменилось и метрика R2 совпадает на стандартных и преобразованных данных, а это означает, что нам удалось защитить данные клиентов и при этом не ухудшить качество модели.

## Вывод:
1. Знакомство с данными:
    - В датасете 5000 строк и 5 столбцов.
    - Весь датасет был переведен в формат `int32`.
    - В датасете были обнаружены и впоследствии удалены 153 дубликата.
    - Пропусков в датасете нет.
2. При изучении было выяснено, что в случае умножения признаков линейной регрессии на обратимую матрицу качество модели не меняется.
3. Для защиты информации умножим матрицу признаков на обратимую матрицу $P$, которая генерируется случайным образом. Алгоритм преобразования:
    1. Создание случайной квадратной обратимой матрицы с количеством столбцов, которое соответствует количеству признаков.
    2. Проверка матрицы на обратимость.
    3. Умножение матрицы признаков на случайную обратимую матрицу.
    4. Применение модели на преобразованных признаках.
    5. Сравнить метрики R2 на исходных признаках и на признаках, умноженных на обратимую матрицу.
4. После преобразования данных путем умножения признаков на матрицу качество линейной регрессии не изменилось и метрика R2 совпадает на стандартных и преобразованных данных, а это означает, что нам удалось защитить данные клиентов и при этом не ухудшить качество модели.